In [2]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

In [3]:
def rhs_fun(x, n, domain_index,result):
    global phi_static
    result[:] = phi_static
    
def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

def energy_mol(mesh_diel, mesh_stern, phi_static, eps_s):

    grid_diel = generate_grid(mesh_diel)
    grid_stern = generate_grid(mesh_stern)

    space_diel = bempp.api.function_space(grid_diel, "DP", 0)
    space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
    N_diel = grid_diel.leaf_view.entity_count(0)
    N_stern = grid_stern.leaf_view.entity_count(0)
    
    elements_d = list(grid_diel.leaf_view.entity_iterator(0))
    elements_s = list(grid_stern.leaf_view.entity_iterator(0))

    phis_grid_fun = bempp.api.GridFunction(space_diel, fun=rhs_fun)

    rhs = numpy.concatenate([eps_s*phis_grid_fun.coefficients, 
                      numpy.zeros(N_stern)])
    

    M11   = laplace.single_layer(space_diel, space_diel, space_diel) 
    M12   = laplace.single_layer(space_stern, space_diel, space_diel) 
    M21   = laplace.single_layer(space_diel, space_stern, space_stern)
    M22   = laplace.single_layer(space_stern, space_stern, space_stern) 

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0,0] = M11
    blocked[0,1] = M12
    blocked[1,0] = M21
    blocked[1,1] = M22
    op_discrete = blocked.strong_form()
    
    sigma, info = gmres(op_discrete, rhs, tol=1e-5, maxiter=500, restart = 1000)
    
    if info>0:
        print 'Not converged, %i iterations'%info
    elif info<0:
        print 'Solver diverges'
    
    sigma_d = sigma[:N_diel]
    sigma_s = sigma[N_diel:]

    
    qe = 1.60217662e-19
    Rw = 1.4
    Na = 6.0221409e23
    eps_0 = 8.854187817e-12
    
    #water_charge = 0.83*qe/(numpy.pi*Rw**2) * Na*qe*1e10/(1000*eps_0)
    water_dipole = 6.17e-30/(1e-10)/((2*Rw)**2) * Na*qe*1e10/(1000*eps_0)
    print water_dipole
    print sigma_s[:5]*1.4
    print max(abs(sigma_s*1.4))
    '''
    test_charge = numpy.where(abs(sigma_d)*1.4>water_dipole)[0]
    #print abs(sigma_d)
    if len(test_charge)>0:
        print 'Unphysical charge in dielectric surface'
        print len(test_charge)
    '''    
    
    test_charge = numpy.where(abs(sigma_s)*1.4>water_dipole)[0]
    if len(test_charge)>0:
        print 'Unphysical charge in stern surface'
        print len(test_charge)
    
    area_d = numpy.zeros(N_diel)
    for i in range(N_diel):
        area_d[i] = elements_d[i].geometry.volume
    
    energy = 0.5*numpy.sum(area_d*sigma_d*phi_static)

    return energy

In [ ]:
phi_static = 10. #kcal/mol/e
phi_static *= 4.184 # kJ/mol/e

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 1.#80.

conv_factor = (1000*eps_0)/(qe**2*Na*1e10)

problem_folder = 'mobley_test/'
namefile = 'actually_all_names.txt'
num_lines = sum(1 for line in open(problem_folder+namefile))
energy = numpy.zeros(num_lines)
it = 0
for line in file(problem_folder+namefile):
    mol = line[:-1]

    mesh_stern = problem_folder+mol+'/surf_d02_stern'
    mesh_diel = problem_folder+mol+'/surf_d02'

    try: 
        e = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
    except:
        print 'Something went wrong'
        e = 0
            
    ekJ = e*conv_factor # kJ/mol
    ekcal = ekJ/4.184   # kcal/mol
    
    energy[it] = ekcal
    
    print 'Run %i for '%it + mol + '. Energy = %1.3f kcal/mol'%energy[it]
    
    it += 1
    
numpy.savetxt('energy_bem.txt', energy)

857.593881087
[  -3.29858506 -256.97958675 -105.87498947  -86.2291428   -61.95141142]
450.149683454
Run 0 for 1112_tetrachloroethane. Energy = 1.700 kcal/mol
857.593881087
[ -16.73538278 -351.49644517 -157.82194318 -114.89586016 -118.49861404]
421.520802
Run 1 for 111_trichloroethane. Energy = 1.573 kcal/mol


In [4]:
mesh_stern = 'mobley_test/1112_tetrachloroethane/surf_d04_stern'
mesh_diel = 'mobley_test/1112_tetrachloroethane/surf_d04'

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 1.#80.

phi_static = 40. #kJ/mol/e


e = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
ekj = e / ((qe**2*Na*1e10)/(1000*eps_0))
ekcal = ekj/4.184
#print e, e / ((qe**2*Na*1e10)/(1000*eps_0)), e/(1000*4.184) #kJ/mol
print ekcal

857.593881087
[-38.81146449 -35.64174868 -32.39337465 -33.00538384 -31.49356311]
438.909780327
1.54818840252


In [33]:
grid_diel = generate_grid(mesh_diel)
grid_stern = generate_grid(mesh_stern)

space_diel = bempp.api.function_space(grid_diel, "DP", 0)
space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
N_diel = grid_diel.leaf_view.entity_count(0)
N_stern = grid_stern.leaf_view.entity_count(0)
    
elements_d = list(grid_diel.leaf_view.entity_iterator(0))
elements_s = list(grid_stern.leaf_view.entity_iterator(0))

print elements_d[0].geometry.corners
print elements_d[0].geometry.local2global(numpy.array([[1.,0.,0.]]))

[[-0.802 -0.885 -0.46 ]
 [ 1.119  0.807  1.406]
 [ 0.339  0.758  0.614]]


AttributeError: 'Geometry' object has no attribute 'global2local'

In [20]:
F = 1000*eps_0/(qe**2*Na*1e10)
1/(4*numpy.pi*F*4.184)
9081457.48434/(332.063*4*numpy.pi)
(qe**2*Na*1e10)/(eps_0)
2353.39653991/40

58.834913497749994

In [ ]:
1.54